# Selección de variables en regresión lineal

En esta sección se exploran los métodos clásicos de selección automática de variables. 

## Preliminares

En este documento se presentan varias alternativas para las selección automática de variables en modelos de regresión. Esta técnicas automáticas resulta útiles cuando nos enfrentamos a gran cantidad de variables y esto hace que el proceso manual sea difícil de abordar. En cualquier caso, hemos de saber que no son mágicas y que tienen sus debilidades, por lo que el control de las mismas por nuestra parte se hace fundamental de cara a la obtención de buenos resultados en su aplicación. 


Procedemos a la lectura de los datos depurados y con las transformaciones creadas en el código de regresión lineal. 


In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Leer datos depurados datosvinoDep
todo_cont = pd.read_csv('C:\\Users\\Guille\\Documents\\MineriaDatos_2022_23\\PARTE I_Depuracion y Regresiones\\Dia2_Regresion Lineal\\todo_cont_cor.csv', index_col=0)

# Descriptivo de comprobación
todo_cont.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 0 to 6364
Data columns (total 30 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Acidez                4998 non-null   float64
 1   AcidoCitrico          4998 non-null   float64
 2   Azucar                4998 non-null   float64
 3   CloruroSodico         4998 non-null   float64
 4   Densidad              4998 non-null   float64
 5   pH                    4998 non-null   float64
 6   Sulfatos              4998 non-null   float64
 7   Alcohol               4998 non-null   float64
 8   CalifProductor        4998 non-null   int64  
 9   PrecioBotella         4998 non-null   float64
 10  Etiqueta              4998 non-null   object 
 11  Clasificacion         4998 non-null   object 
 12  Region                4998 non-null   float64
 13  prop_missings         4998 non-null   float64
 14  Etiqueta2             4998 non-null   object 
 15  aleatorio            

 ### Preparación de los datos
 
 Como siempre sacamos la variable objetivo para tenerla controlada y creamos el input. Como en esta ocasión vamos a trabajar más con el paradigma modelización mediante X,y, necesitaremos generar explítcitamente la matriz de diseño total con las categóricas extendidas en dummies y con constante. Vamos a hacerlo de forma manual. 
 

In [2]:
varObjCont = todo_cont.Beneficio
imput = todo_cont.drop(['Beneficio'],axis=1)

# Craer matriz de diseño 
imput_dummy = pd.get_dummies(imput, columns=['Clasificacion', 'Etiqueta', 'Region'], drop_first=False)

# Borramos los niveles que queramos como referencia (se incluirá su efecto implicito en las constante)
imput_dummy.drop(['Etiqueta_MM','Clasificacion_*','Region_1.0'], axis=1, inplace=True)
# Añadir constante
imput_dummy=sm.add_constant(imput_dummy)

imput_dummy.head()

,const,Acidez,AcidoCitrico,Azucar,CloruroSodico,Densidad,pH,Sulfatos,Alcohol,CalifProductor,...,Clasificacion_**,Clasificacion_***,Clasificacion_****,Clasificacion_Desc,Etiqueta_B,Etiqueta_M,Etiqueta_MB,Etiqueta_R,Region_2.0,Region_3.0
0,1.0,0.16,-0.81,26.10,-0.425,1.02792,3.38,0.70,15.4,2,...,0,1,0,0,0,1,0,0,0,0
1,1.0,2.64,-0.88,14.80,0.037,0.99518,3.12,0.48,22.0,3,...,0,1,0,0,0,1,0,0,0,1
3,1.0,-1.22,0.34,1.40,0.040,1.03236,3.20,0.33,11.6,2,...,0,1,0,0,1,0,0,0,1,0
4,1.0,0.27,1.05,11.25,-0.007,0.99620,4.93,0.26,15.0,1,...,0,0,0,1,0,0,0,1,1,0
5,1.0,-0.22,0.39,1.80,-0.277,0.94724,3.09,0.75,12.6,3,...,0,0,1,0,0,0,0,1,1,0


Tomamos las particiones de training y test desde la matriz de diseño.

In [3]:
# Función necesaria
from sklearn.model_selection import train_test_split

# Creamos 4 objetos: predictores para tr y tst y variable objetivo para tr y tst. 
X_train, X_test, y_train, y_test = train_test_split(imput_dummy, varObjCont, test_size=0.2, random_state=42)

# Comprobamos dimensiones
print('Training dataset shape:', X_train.shape, y_train.shape)
print('Testing dataset shape:', X_test.shape, y_test.shape)

Training dataset shape: (3998, 37) (3998,)
Testing dataset shape: (1000, 37) (1000,)


### Modelo con todos los efectos

Ajsutamos un modelo con todos los efectos que, aunque inutil por sus múltiples problemas de colinealidad, sobreparametrización etc, nos sirve para controlar como está la cosa con las variables. 


In [4]:
# Importamos la api para fórmulas (en concreto ols para regresión)
from statsmodels.formula.api import ols 
import statsmodels.api as sm

# Genero el training con la objetivo dentro 
data_train = X_train.join(y_train)

res = sm.OLS(np.asarray(y_train),X_train).fit()
res.summary()


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).


## Modelo manual ganador

Rescatamos el modelo ganador en nuestro proceso de ajuste manual de modelos de regresión lineal.

In [5]:
# Ajusto regresión de ejemplo
results = ols('Beneficio ~ Etiqueta + Clasificacion + CalifProductor + Acidez + Alcohol',data=todo_cont).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Beneficio   R-squared:                       0.434
Model:                            OLS   Adj. R-squared:                  0.433
Method:                 Least Squares   F-statistic:                     348.2
Date:                Sat, 12 Nov 2022   Prob (F-statistic):               0.00
Time:                        12:21:55   Log-Likelihood:                -32696.
No. Observations:                4998   AIC:                         6.542e+04
Df Residuals:                    4986   BIC:                         6.549e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               643.8888     11.597     55.520      0.000     621.153     666.625
Etiqueta[T.M]          -261.4073      7.246    -36.077      0.000    -275.612    -247.202
Etiqueta[T.MB]          109.7725     13.567      8.091      0.000      83.175     136.370
Etiqueta[T.MM]         -384.7397     13.117    -29.331      0.000    -410.455    -359.024
Etiqueta[T.R]          -129.0588      6.249    -20.654      0.000    -141.309    -116.808
Clasificacion[T.**]      45.7919      6.381      7.176      0.000      33.282      58.302
Clasificacion[T.***]    104.1102      7.326     14.212      0.000      89.749     118.472
Clasificacion[T.****]   188.2680     11.028     17.073      0.000     166.649     209.887
Clasificacion[T.Desc]   -21.2381      8.105     -2.621      0.009     -37.127      -5.350
CalifProductor           -8.4614      2.216     -3.818      0.000     -12.806      -4.117
Acidez                   -7.4238      3.108     -2.389      0.017     -13.516      -1.331
Alcohol                   4.0767      0.669      6.091      0.000       2.765       5.389
==============================================================================
Omnibus:                      132.062   Durbin-Watson:                   2.006
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              142.233
Skew:                           0.413   Prob(JB):                     1.30e-31
Kurtosis:                       3.044   Cond. No.                         80.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Selección automática de variables 

Vamos a probar ahora los métodos clásicos de selección de variables que, partiendo del modelo completo/nulo eliminarán/añadirán secuencialmente variables hasta un número indicado o bien hasta alcanzar el score mejor o de mayor parsimonia. 

In [6]:
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as sfs

clf = LinearRegression()

# Build step forward feature selection
sfs_back = sfs(clf,k_features = 'best',forward=False,floating=False, scoring='r2',cv=5)

# Perform SFFS
sfs_back = sfs_back.fit(X_train, y_train)

#print(sfs1.subsets_)

print(sfs_back.k_feature_names_)

sfs_back.k_score_

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Guille\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Guille\anaconda3\lib\site-packages\sklearn\linear_model\_base.py", line 684, in fit
    X, y = self._validate_data(
  File "C:\Users\Guille\anaconda3\lib\site-packages\sklearn\base.py", line 596, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\Guille\anaconda3\lib\site-packages\sklearn\utils\validation.py", line 1074, in check_X_y
    X = check_array(
  File "C:\Users\Guille\anaconda3\lib\site-packages\sklearn\utils\validation.

In [ ]:
pd.DataFrame.from_dict(sfs_back.get_metric_dict()).T

In [ ]:
# Sequential Forward Selection
sfs_forw = sfs(clf, 
          k_features='parsimonious', 
          forward=True, 
          floating=False, 
          scoring='r2',
          cv=4)

sfs_forw = sfs_forw.fit(X_train, y_train)

print('\nSequential Backward Selection:')
print(sfs_forw.k_feature_names_)
print('CV Score:')
print(sfs_forw.k_score_)

In [ ]:
# Proceso forward
pd.DataFrame.from_dict(sfs_forw.get_metric_dict()).T

### Visualicación del proceso de selección de variables

In [ ]:
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
import matplotlib.pyplot as plt

fig1 = plot_sfs(sfs_forw.get_metric_dict(), kind='std_dev')

#plt.ylim([0.8, 1])
plt.title('Sequential Forward Selection (w. StdDev)')
plt.grid()
plt.show()

In [ ]:
# Sequential Forward Selection
sfs_12 = sfs(clf, 
          k_features= 12, 
          forward=False, 
          floating=True, 
          scoring='r2',
          cv=4)

sfs_12 = sfs_12.fit(X_train.drop(['aleatorio2_log'],axis=1), y_train)

print('\nSequential Forward Selection (k=12):')
print(sfs_12.k_feature_names_)
print('CV Score:')
print(sfs_12.k_score_)


In [ ]:
# Sequential Forward Selection
sfs_10 = sfs(clf, 
          k_features= 10, 
          forward=False, 
          floating=True, 
          scoring='r2',
          cv=4)

sfs_10 = sfs_10.fit(X_train.drop(['aleatorio2_log'],axis=1), y_train)

print('\nSequential Forward Selection (k=10):')
print(sfs_10.k_feature_names_)
print('CV Score:')
print(sfs_10.k_score_)


## Comparación por validación cruzada

Comparamos el rendimiento de los modelos bajo el esquema de validación cruzada repetida creando una función similar a la que ya teníamos pero que, en esta ocasión trabaja sobre objetos de salida de los métodos de selección de variables de tal forma que en base a estos se seleccione el input adecuado y se ajuste el modelo lineal con las varibales seleccionadas. 


In [ ]:
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression

model =LinearRegression()

# Función para comparación por validación cruzada
def cross_val (sfs, data, y, seed=12345):
        
        X = sfs

        if not isinstance(sfs,pd.DataFrame):
            X = sfs.transform(data)

        # Establecemos esquema de validación fijando random_state (reproducibilidad)
        cv = RepeatedKFold(n_splits=5, n_repeats=20, random_state=seed)

        # Obtenemos los resultados de R2 para cada partición tr-tst
        scores = cross_val_score(model, X, y, cv=cv, n_jobs=-1)

        # Sesgo y varianza
        print('Coeficiente de determinación R2: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

       # sns.violinplot(y=scores,palette='viridis')

        return(scores)


In [ ]:
# Ejemplo de uso par aun modelo
# cross_val(sfs_back,imput_dummy,varObjCont)

In [ ]:
# Creamos lista de fórmulas   
list_sfs = [sfs_back,sfs_forw,sfs_12,sfs_10]
list_sfs

# Aplicamos a toda la lista la función creada (devuelve un dataframe pero está transpuesto)
list_res = pd.DataFrame(map(lambda x: cross_val(x,imput_dummy,varObjCont, seed=2022),list_sfs))

# Trasnponer dataframe y pasar de wide a long (creando un factor variable con el nombre de cada fórmula de la lista[0,1,2,3])
results = list_res.T.melt()
results.columns = ['Modelo','R2']
results.head()


In [ ]:
# Boxplot paralelo para comparar
sns.boxplot(x='Modelo',y='R2',data=results,palette='viridis')

## Selección de variables con interacciones

Vamos ahora a considerar los efectos de interacción de orden 2 entre las variables para valorar si pueden aportar capacidad predictiva al modelo. 

Generaremos el dataset con las interacciones de todas las variables y posteriormente pasaremos los métodos de selección para hacer una criba de efectos interesantes. 

In [ ]:
sel_col = ['const', 'Acidez', 'CalifProductor', 
        'Acidez_sqr', 'Alcohol_raiz4', 'Clasificacion_**', 'Clasificacion_***', 'Clasificacion_****', 
        'Clasificacion_Desc', 'Etiqueta_B', 'Etiqueta_M', 'Etiqueta_MB', 
        'Etiqueta_R', 'Region_2.0']

imput_red = imput_dummy[sel_col]
X_train_red = X_train[sel_col]

In [ ]:
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures

# Create interaction terms (interaction of each regressor pair + polynomial)
#Interaction terms need to be created in both the test and train datasets
interaction = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
interaction

#traning
imput_inter = pd.DataFrame(interaction.fit_transform(imput_red), columns=interaction.get_feature_names_out(input_features=imput_red.columns))
X_inter = pd.DataFrame(interaction.fit_transform(X_train_red), columns=interaction.get_feature_names_out(input_features=X_train_red.columns))
X_inter.head(3)

In [ ]:
# Eliminar columnas constantes (interacciones sin sentido)
X_inter = X_inter.loc[:, X_inter.var() != 0.0]

X_inter.head(3)

In [ ]:
# Sequential Forward Selection
sfs_forw_int_10 = sfs(clf, 
          k_features=10, 
          forward=True, 
          floating=False, 
          scoring='r2',
          cv=4)

sfs_forw_int_10 = sfs_forw_int_10.fit(X_inter, y_train)

print('\nSequential Backward Selection:')
print(sfs_forw_int_10.k_feature_names_)
print('CV Score:')
print(sfs_forw_int_10.k_score_)


In [ ]:
# Sequential Forward Selection
sfs_forw_int_best = sfs(clf, 
          k_features='best', 
          forward=True, 
          floating=False, 
          scoring='r2',
          cv=4)

sfs_forw_int_best = sfs_forw_int_best.fit(X_inter, y_train)

print('\nSequential Backward Selection:')
print(sfs_forw_int_best.k_feature_names_)
print('CV Score:')
print(sfs_forw_int_best.k_score_)


## Selección de variables por LASSO

Exploramos la selección de variables por modelo laso con criterios AIC o BIC.

In [ ]:
from sklearn import linear_model

reg = linear_model.LassoLarsIC(criterion='bic', normalize=False)

reg.fit(X_train, y_train)

print(reg.coef_)

In [ ]:
selec_feats = X_train[X_train.columns[(reg.coef_ != 0).ravel().tolist()]]
selec_feats

Lasso con interacciones

In [ ]:
lasso_int = linear_model.LassoLarsIC(criterion='bic', normalize=False)

lasso_int.fit(X_inter, y_train)

print(lasso_int.coef_)

In [ ]:
selec_feats_int = X_inter[X_inter.columns[(lasso_int.coef_ != 0).ravel().tolist()]]
selec_feats_int

### Validación Cruzada

In [ ]:
list_sfs = [sfs_forw_int_10,sfs_forw_int_best,selec_feats,selec_feats_int]
list_sfs

data = imput_dummy.join(imput_inter,lsuffix="_left")

# Aplicamos a toda la lista la función creada (devuelve un dataframe pero está transpuesto)
list_res = pd.DataFrame(map(lambda x: cross_val(x,X_inter,y_train, seed=2022),list_sfs))

# Trasnponer dataframe y pasar de wide a long (creando un factor variable con el nombre de cada fórmula de la lista[0,1,2,3])
results = list_res.T.melt()
results.columns = ['Modelo','R2']
results.head()

In [ ]:
# Boxplot paralelo para comparar
sns.boxplot(x='Modelo',y='R2',data=results,palette='viridis')